# **Punctuation Restoration in the Russian Language using pre-trained RoBERTa model with CRF-head**

# Preprocessing

For training dataset from russian N+1 magazine was chosen. Only period and comma symbols were kept. 

In [1]:
!pip install pytorch-crf
!pip install deeppavlov
!pip install transformers


import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

!pip install razdel

import razdel
import random
import math
import time

from torchtext.data import Field

import torch.nn.functional as F

     |████████████████████████████████| 849kB 2.7MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 8.0MB 12.9MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 1.5MB 43.5MB/s 
     |████████████████████████████████| 7.3MB 10.1MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 2.1MB 58.2MB/s 
     |████████████████████████████████| 6.7MB 61.3MB/s 
     |████████████████████████████████| 10.4MB 55.5MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 512kB 51.3MB/s 
     |████████████████████████████████| 665kB 54.6MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 860kB 30.3MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     

     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 14.0MB/s 
     |████████████████████████████████| 3.0MB 20.3MB/s 


In [2]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

from google.colab import drive
drive.mount('/content/drive')

%cd 'drive/My Drive'

cuda
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [3]:
from razdel import tokenize
import re


with open('nplus.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    lines = [re.sub("[`~*)@#%(&$_^]’", '', line) for line in lines]
    lines = [re.sub("\d+", 'число', line) for line in lines]
    lines = [line.replace('«','') for line in lines]
    lines = [line.replace('»','') for line in lines]
    lines = [line.replace('#','') for line in lines]
    lines = [line.replace('@','') for line in lines]
    lines = [line.replace('_','') for line in lines]
    lines = [line.replace('/','') for line in lines]
    lines = [line.replace('\\','') for line in lines]
    lines = [line.replace('&','') for line in lines]
    lines = [line.replace('—','') for line in lines]
    lines = [line.replace('–','') for line in lines]
    lines = [line.replace('-',' ') for line in lines]
    lines = [line.replace('…','.') for line in lines]
    lines = [line.replace(';',',') for line in lines]
    lines = [line.replace(')','') for line in lines]
    lines = [line.replace('(','') for line in lines]
    lines = [line.replace('?','.') for line in lines]
    lines = [line.replace('!','.') for line in lines]
    lines = [line.replace(':',',') for line in lines]
    lines = [line.replace('„',',') for line in lines] 
    lines = [line.replace('*','') for line in lines] 
    lines = [line.replace("'",'') for line in lines] 
    lines = [line.replace('"','') for line in lines] 
    lines = [line.replace('...','.') for line in lines] 
    lines = [line.replace('..','') for line in lines] 
    lines = [line.strip() for line in lines]
    lines_rus = [line for line in lines if line]

    print(lines_rus[:10])
    

lines = lines_rus
max_len = 80
corpus = []

for n, i in enumerate(lines):
    
    string = [_.text for _ in list(tokenize(i))]
    if 5 <= len(string) <= max_len:
        corpus.append(string)
    else:
        pass


    if (n%1000) == 0:
        print(n)

test = corpus[1800:]
corpus = corpus[:1800]

print(corpus[0])
print(corpus[1])


['Сотрудники Венского медицинского университета разработали технологию, которая позволяет пациентам с травмой плечевого нервного сплетения управлять бионическим протезом. Работа опубликована\xa0в журнале Lancet. Плечевое сплетение дает начало нервам, которые управляют движением всех мышц руки. Травмы этого сплетения приводят к функциональной ампутации конечности, причем восстановить ее иннервацию обычно не удается. Вместо попыток наладить иннервацию австрийские медики решили использовать бионический протез. Управлять им должна была остаточная активность нервов плеча, а также работа нервов, ранее пересаженных пациентам вместе с мышцей с другой части тела ноги. Обнаружив слабую активность плечевых нервов, ученые снабдили их электрическими сенсорами и начали тренировать пациентов управлять виртуальной рукой, которую показывали на экране компьютера. После девяти месяцев тренировок электрическая активность нервных окончаний значительно возросла, и к нервам подключили бионический протез. Пер

In [4]:
from sklearn.model_selection import train_test_split
import string

print(len(corpus))
print(len(test))

TRG = Field(init_token = '<sos>', 
            eos_token = '<eos>')

1800
145


# Source- and target-sequences preparation

In [5]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs
import transformers
from transformers import BertTokenizer
from transformers import BertModel


punct = [',','.','?','!']




X = [[c for c in s if c not in punct] for s in corpus]

Y = []
dir = 'rubert_cased_L-12_H-768_A-12_pt'
tokenizer = BertTokenizer.from_pretrained(dir, do_lower_case = False, max_length = 100, model_max_length = 100)

corpus = [' '.join(i) for i in corpus]
for snt in corpus:
    
    src = tokenizer([snt], return_tensors="pt", padding='max_length')

    res = []
    p = {128:',', 132:'.'}
    r = src['input_ids'][0]
    for i in range(len(r)-1):

        if r[i].item() not in p.keys():
            if r[i+1].item() in p:
                res.append(p[r[i+1].item()])
            else:
                res.append('_')
    Y.append(res)



n = 23
ans1 = X[n]
ans = Y[n]
print('SRC:', ans1)
print('TRG:', ans)



print(X[:10])
X = [' '.join(i) for i in X]
print(X[:10])

train_src, valid_src, train_trg, valid_trg = train_test_split(X, Y, test_size=0.2, shuffle = True, random_state=seed)

l = [train_src, valid_src, train_trg, valid_trg]
for i in l:
  print(len(i))



TRG.build_vocab([i for i in Y], min_freq = 10)
print("Number of tokens in vocabulary", len(TRG.vocab))
print(TRG.vocab.stoi.items())

SRC: ['Управление', 'научно', 'исследовательских', 'работ', 'ВМС', 'США', 'провело', 'технические', 'испытания', 'нового', 'поколения', 'беспилотников', 'Об', 'этом', 'сообщается', 'на', 'сайте', 'военно', 'морского', 'министерства', 'США']
TRG: ['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '.', '_', '_', '_', '_', '_', '_', '_', '_', '.', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']
[['Компания', 'DigitalGlobe', 'опубликовала', 'первые', 'коммерчески', 'доступные', 'снимки', 'с', 'разрешением', 'в', 'число', 'сантиметр', 'Ранее', 'американские', 'компании', 'могли', 'продавать', 'снимки', 'с', 'разрешением', 'меньше', 'полуметра', 'только', 'прав

## Custom dataset-class


In [16]:
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_sequence
from torch.nn.utils.rnn import PackedSequence
from torch.nn.utils.rnn import pad_packed_sequence
from torch.nn.utils.rnn import pad_sequence


class PunctDataset(Dataset):
    def __init__(self, src_data, trg_data):

        self.src = src_data
        self.trg = trg_data

        print(self.src[0])
        print(self.trg[0])
        
    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        
        return self.src[idx], [TRG.vocab.stoi[i] for i in self.trg[idx]] + [TRG.vocab.stoi['<eos>']]
        
    def collate_fn(self, batch):
        
        return [pair[0] for pair in batch], pad_packed_sequence(pack_sequence([torch.tensor(pair[1]) for pair in batch], enforce_sorted=False), padding_value = TRG.vocab.stoi['<pad>'], total_length = 100, batch_first = True)[0]


## Model based on RoBERTa and CRF-head


Neural networks with CRF-heads are promising solutions for the tagging task (punstuation restoration is the same task) Links: https://www.semanticscholar.org/paper/Vietnamese-Punctuation-Prediction-Using-Deep-Neural-Pham-Nguyen/eca555c0950bd048adaf187f75ede9ac2248e672, http://www.nwpu-aslp.org/lxie/papers/2016ISCSLP-XKT.pdf

I used pre-trained RoBERTa model from DeepPavlov. CRF-modul was imported from pytorch-crf package


In [25]:
from torchcrf import CRF

class PunctCorrector(nn.Module):
    def __init__(self, num_cls):
        """
        :param: num_cls is the size of target space

        """
        super().__init__()
        
        self.num_cls = num_cls

        self.ln = nn.Linear(768, 256)
        self.ln2 = nn.Linear(256, num_cls)
        
        self.model_CRF = CRF(num_cls, batch_first = True)

        torch.nn.init.constant_(self.model_CRF.start_transitions[3], -1e9) # to prevent EOS-tag transitions
        torch.nn.init.constant_(self.model_CRF.transitions[3], -1e9)


    def forward(self, embedded, trg = None, mask = None, mode = 'trn'):
        """
        :param: src sentences
        :param: trg sentences
        :param: mask of trg for CFR
        :param: mode - train or eval

        """

        outputs = self.ln2(F.relu(self.ln(embedded)))

        if mode == 'trn':

            if mask != None:
                loss = -self.model_CRF(outputs, trg, mask=mask)
            else:
                loss = -self.model_CRF(outputs, trg)
            out = self.model_CRF.decode(outputs)

            return loss, out

        if mode == 'eval':
             out = self.model_CRF.decode(outputs)

             return out

In [26]:
number_classes = len(TRG.vocab)
batch_size_trn = 64
batch_size_eval = 256



dataset_trn = PunctDataset(train_src, train_trg)
dataloader_trn = DataLoader(dataset_trn, 
                              batch_size=batch_size_trn,
                              shuffle=True,
                              collate_fn=dataset_trn.collate_fn)

dataset_val = PunctDataset(valid_src, valid_trg)
dataloader_val = DataLoader(dataset_val, 
                              batch_size=batch_size_eval,
                              shuffle=True,
                              collate_fn=dataset_val.collate_fn)



model = PunctCorrector(number_classes).to(device)

dir = 'rubert_cased_L-12_H-768_A-12_pt'
tokenizer = BertTokenizer.from_pretrained(dir, do_lower_case = False, max_length = 100, model_max_length = 100)
roberta = BertModel.from_pretrained(dir).to(device)

Израильские и венгерские ученые обнаружили генетическую
['_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_']
Итальянские физики при поддержке Национального института ядерной физики в Генуе разработали алмазный детектор для измерения потока нейтронов Устройство которое может найти применение в термоядерных реакторах было описано в докладе на конференции ANIMMA число с его содержанием можно ознакомиться на сайте arXiv org Авторы работы предложили использовать для нейтронного детектора алмазы выращенные методом химического осаждения из газовой фазы В отл

In [27]:
print(model)

total_params = sum(p.numel() for p in model.parameters()if p.requires_grad)
print('Number of trainable parameters: ', total_params)
print('\n')

PunctCorrector(
  (ln): Linear(in_features=768, out_features=256, bias=True)
  (ln2): Linear(in_features=256, out_features=7, bias=True)
  (model_CRF): CRF(num_tags=7)
)
Number of trainable parameters:  198726




In [28]:
def train(tokenizer, roberta, model, iterator, optimizer):
    
    model.train()
    roberta.eval()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        model.train()
        
        src = batch[0]
        trg = batch[1].to(device)

        mask = trg.clone().detach()
        mask[mask==1] = 0
        mask[mask!=0] = 1
        mask = torch.tensor(mask, dtype=torch.uint8).to(device)

        
        optimizer.zero_grad()

        src = tokenizer(src, return_tensors="pt", padding='max_length')

        with torch.no_grad():
            embedded = roberta.forward(**src.to(device))[0]

        loss, output = model(embedded, trg, mask)

        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [29]:
def evaluate(tokenizer, roberta, model, iterator):
    
    model.eval()
    roberta.eval()

    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1].to(device)

            mask = trg.clone().detach()
            mask[mask==1] = 0
            mask[mask!=0] = 1
            mask = torch.tensor(mask, dtype=torch.uint8).to(device)

            src = tokenizer(src, return_tensors="pt", padding='max_length')
            
            embedded = roberta.forward(**src.to(device))[0]   

            loss, output = model(embedded, trg, mask)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


In [30]:
max_epochs = 25
lr = 1e-2
weight_decay = 5e-6
step_size = 3
gamma = 0.9

message = 'Training starts... ' + '\nNumber trn batches: '+str(len(dataloader_trn)) + '\nNumber eval batches: '+str(len(dataloader_val))


optimizer = optim.Adam(model.parameters(), lr = lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)




print(message)
best_valid_loss = float('inf')
for epoch in range(1, max_epochs+1):

    train_loss = round(train(tokenizer, roberta, model, dataloader_trn, optimizer), 5)
    valid_loss = round(evaluate(tokenizer, roberta, model, dataloader_val), 5)
    scheduler.step()
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
        print("Saving best model's weights")
        
    

    print(f"| EPOCH №{epoch:3.0f} | TRN LOSS {train_loss:.5f} | VAL LOSS {valid_loss:.5f} | LR {scheduler.get_last_lr()[0]:.7f}")

Training starts... 
Number trn batches: 23
Number eval batches: 2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Saving best model's weights
| EPOCH №  1 | TRN LOSS 1959.51971 | VAL LOSS 2940.83655 | LR 0.0100000
Saving best model's weights
| EPOCH №  2 | TRN LOSS 520.28565 | VAL LOSS 1073.24597 | LR 0.0100000
Saving best model's weights
| EPOCH №  3 | TRN LOSS 295.32509 | VAL LOSS 832.84433 | LR 0.0090000
Saving best model's weights
| EPOCH №  4 | TRN LOSS 225.43429 | VAL LOSS 722.77420 | LR 0.0090000
Saving best model's weights
| EPOCH №  5 | TRN LOSS 192.52420 | VAL LOSS 676.25943 | LR 0.0090000
Saving best model's weights
| EPOCH №  6 | TRN LOSS 170.71404 | VAL LOSS 657.10431 | LR 0.0081000
Saving best model's weights
| EPOCH №  7 | TRN LOSS 156.89725 | VAL LOSS 641.17300 | LR 0.0081000
Saving best model's weights
| EPOCH №  8 | TRN LOSS 137.86074 | VAL LOSS 608.16797 | LR 0.0081000
Saving best model's weights
| EPOCH №  9 | TRN LOSS 125.36949 | VAL LOSS 595.86343 | LR 0.0072900
| EPOCH № 10 | TRN LOSS 113.54356 | VAL LOSS 604.03864 | LR 0.0072900
| EPOCH № 11 | TRN LOSS 106.91408 | VAL LOSS 

In [31]:
import string
from nltk.tokenize import wordpunct_tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize



def check_one(reference, hypothesis):
    correct = 0
    incorrect = 0
    ref = wordpunct_tokenize(reference)
    hyp = wordpunct_tokenize(hypothesis)
    ref_i, hyp_i = 0, 0
    punct_places = 1e-7
    while ref_i < len(ref) and hyp_i < len(hyp):
        need_punct_check_ref = False
        need_punct_check_hyp = False
        cur_ref = ref[ref_i]
        if cur_ref in string.punctuation:
            need_punct_check_ref = True
            punct_places += 1
        cur_hyp = hyp[hyp_i]
        if cur_hyp in string.punctuation:
            need_punct_check_hyp = True
        if need_punct_check_ref and need_punct_check_hyp:
            if cur_ref == cur_hyp:
                correct += 1
            else:
                incorrect += 1
            ref_i += 1
            hyp_i += 1
            continue

        if need_punct_check_ref and not need_punct_check_hyp:
            incorrect += 1
            ref_i += 1
            continue

        if not need_punct_check_ref and need_punct_check_hyp:
            incorrect += 1
            hyp_i += 1
            continue

        assert cur_hyp == cur_ref, "The phrases are inconsistent!"
        ref_i += 1
        hyp_i += 1

    if punct_places == 1e-7:
        if correct == 0:
            if incorrect == 0:
                return 1
            else:
                return correct/punct_places - incorrect/(2 * len(reference))
        else:
            return correct/punct_places - incorrect/(2 * len(reference))
    else:
         return correct/punct_places - incorrect/(2 * len(reference))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
model.load_state_dict(torch.load('model.pt'))




data = test
if not isinstance(data[0], str):
    data = [' '.join(i) for i in data]

sentences = data
#sentences = ['''Уникальный макет очень точно копирует центральную часть города. В нем воссозданы мозаики, барельефы, дорожная разметка, подземные переходы и даже пандусы.''']


q = 0
for n, sentence_gold in enumerate(sentences):

    sentence = " ".join(filter(lambda x: x not in string.punctuation, word_tokenize(sentence_gold)))

    sentence_token = tokenizer(sentence, return_tensors="pt", padding='max_length')
    sentence_token = tokenizer.convert_ids_to_tokens(sentence_token['input_ids'][0])
    sentence_token = [i for i in sentence_token if i != '[PAD]']
   

    src = tokenizer(sentence, return_tensors="pt", padding='max_length')
    embedded = roberta.forward(**src.to(device))[0]   
    output = model(embedded, mode = 'eval')

    ans = []
    for t in output[0]:
            ans.append(TRG.vocab.itos[t])

    
    res = ''
    if ans[0] == '_':
        pass
    else:
        res += ans[0]
        res += ' '

    for i in range(1, len(sentence_token)-1):

                if ans[i] == '_':
                    if '##' in sentence_token[i]:
                        res += sentence_token[i].replace("#", "")
                        
                    else:
                        res += ' '
                        res += sentence_token[i]                                       
                        
                else: 
                    if '##' in sentence_token[i]:
                        res += sentence_token[i].replace("#", "")
                        if '##' not in sentence_token[i+1]:
                            res += ans[i]
                        else:
                            pass
                    else:    
                        res += ' '          
                        res += sentence_token[i]
                        if '##' not in sentence_token[i+1]:
                            res += ans[i]
                        else:
                            pass
                        
    

    q += check_one(sentence_gold, res) * 100

    print(sentence_gold)
    #print(sentence_token)
    #print(ans)
    print(res)
    print(check_one(sentence_gold, res) * 100)
    

print("Quality: {:.2f}%".format(q / n+1))
print("Number of sentences in quality calcs: {}.".format(n+1))

Компьютерное моделирование показало , что условия на Проксиме Центавра b могут быть пригодны для существования жизни , если планета имеет атмосферу по плотности схожую с земной . В таком случае газовая оболочка экзопланеты сможет защитить ее обитателей от губительного воздействия излучения центральной звезды . Статья опубликована в журнале Monthly Notices of the Royal Astronomical Society , Letters .
 Компьютерное моделирование показало, что условия на Проксиме Центавра b могут быть пригодны для существования жизни, если планета имеет атмосферу по плотности схожую с земной. В таком случае газовая оболочка экзопланеты сможет защитить ее обитателей от губительного воздействия излучения центральной звезды. Статья опубликована в журнале Monthly Notices of the Royal Astronomical Society Letters.
83.20926246553628
Инженеры из Университета штата Мичиган разработали наногенератор для автономных самозарядных устройств , он позволяет использовать энергию прикосновения к сенсорному экрану для его